# Análise exploratória para a avaliação de recursos da Linha 010

Código feito por Rodrigo Cunha e Fernanda Scovino, ago/23

---

## Objetivo

Identificar viagens circulares não cobertas pela metodologia atual.

## Contexto

- serviço avaliado: 010

<!--  

### Contexto

- Apurar o recurso `SMTR202211008775`:

```
Bom dia, Solicitamos a verificação do itinerário da linha 010, por ser uma linha do tipo circular e com 2 pontos finais, verificamos que diversas viagens não estão sendo apuradas, solicitamos a verificação.
```
-->


## Etapas

- Análise exploratória: Qual o problema?
- Proposta de solução: Como vamos resolver?
- Resultados: Foi resolvido?

## Método

- Métrica de avaliação:
  - Percentual de Operação Diário (POD) -> Expectativa: Mais viagens válidas = aumento do POD, acima de 80% (% de distância excutado / planejado or dia)
  
- Amostra:
  - Opção 1: Dias com pior percentual + dias com melhor percentual
  - Opção 2: Aleatorizar as datas

## Importação de pacotes, configurações e funções

In [33]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.express as px

import folium
from datetime import timedelta, datetime
from shapely import wkt

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve()

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'

for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"


def map_generate(servico, 
                 sentido, 
                 data, 
                 id_viagem = None, 
                 datetime_partida = None, 
                 datetime_chegada = None, 
                 mode = "prod", 
                 table_id_gps = "registros_status_viagem",
                 id_veiculo = None,
                 dataset_id = None):

    if mode == "prod":
        project_id = "rj-smtr"
        if dataset_id is None:
            dataset_id = "projeto_subsidio_sppo"
    elif mode == "dev":
        project_id = "rj-smtr-dev"
        if dataset_id is None:
            dataset_id = "20230815_projeto_subsidio_sppo_circulares"

    title_html = f"""
                <h3 align='center' style='font-size:16px'><b>Serviço: {servico} - Horário: {datetime_partida} a {datetime_chegada}</b></h3>
                """

    f = folium.Figure(width=800, height=600)

    mapa_viagem = folium.Map(
        location=[-22.908333, -43.396388],
        zoom_start=11,
        width=800,
        height=500,
    ).add_to(f)

    mapa_viagem.get_root().html.add_child(folium.Element(title_html))

    colors = ["#fc0000", "#00b7ff", "#00FF00"]

    feature_start_pt = folium.FeatureGroup(name="start_pt")
    feature_end_pt = folium.FeatureGroup(name="end_pt")

    if id_viagem is not None:
        id_viagem_cond = f"AND id_viagem = '{id_viagem}'"
    else:
        id_viagem_cond = ""
        
    status_colors = {'start': '#21DE4D',
                     'middle': '#2CDEBA',
                     'end': '#37A5DE',
                     'out': '#DE4350'}

    if datetime_partida is not None and datetime_chegada is not None:
        date_cond = f"AND timestamp_gps BETWEEN '{datetime_partida}' AND '{datetime_chegada}'"
    else:
        date_cond = ""
        
    if id_veiculo is not None:
        id_veiculo_cond = f"AND id_veiculo = '{id_veiculo}'"
    else:
        id_veiculo_cond = ""

    if table_id_gps == "gps_sppo":
        q = f"""
        SELECT
          timestamp_gps,
          'middle' AS status_viagem,
          ST_GEOGPOINT(longitude, latitude) AS posicao_veiculo_geo
        FROM
          `rj-smtr.br_rj_riodejaneiro_veiculos.{table_id_gps}`
        WHERE
          data = '{data}'
          AND servico = '{servico}'
          {date_cond}
          {id_veiculo_cond}
        """
        
    else:
        q = f"""
        SELECT
          timestamp_gps,
          posicao_veiculo_geo,
          status_viagem
        FROM
          `{project_id}.{dataset_id}.{table_id_gps}`
        WHERE
          data = '{data}'
          AND servico_informado = '{servico}'
          {id_viagem_cond}
          {date_cond}
          {id_veiculo_cond}
        """

    print(q)

    registros_viagem = bd.read_sql(q)
    registros_viagem = registros_viagem.drop_duplicates(["posicao_veiculo_geo", "timestamp_gps"])
    registros_viagem = registros_viagem.sort_values('timestamp_gps')
    registros_viagem["posicao_veiculo_geo"] = registros_viagem["posicao_veiculo_geo"].apply(wkt.loads)

    feature_gps_radius = folium.FeatureGroup(name="raio_gps")
    feature_gps = folium.FeatureGroup(name="gps")

    for _, p in registros_viagem.iterrows():

        folium.Circle(
            location=[
                p.posicao_veiculo_geo.y,
                p.posicao_veiculo_geo.x
            ],
            radius=500,
            weight=1,
            color=p.status_viagem.replace(p.status_viagem, status_colors[p.status_viagem]),
            tooltip=f"{p.timestamp_gps} ({p.status_viagem})",
        ).add_to(feature_gps_radius)

        folium.Circle(
            location=[
                p.posicao_veiculo_geo.y,
                p.posicao_veiculo_geo.x
            ],
            radius=50,
            fill=True,
            color=p.status_viagem.replace(p.status_viagem, status_colors[p.status_viagem]),
            tooltip=f"{p.timestamp_gps} ({p.status_viagem})",
        ).add_to(feature_gps)

    feature_gps.add_to(mapa_viagem)
    feature_gps_radius.add_to(mapa_viagem)

    if sentido is None:
        sentido = "" 
        sentido_cond = ""  
    else:
        sentido_cond = f"AND sentido = '{sentido}'"
        
    q = f"""
    SELECT
      *
    FROM
      `{project_id}.{dataset_id}.viagem_planejada`
    WHERE
      servico = '{servico}'
      AND data = '{data}'
      {sentido_cond}
    """
    
    print(q)
    
    shapes = bd.read_sql(q)

    for (k, t_shape), t_color in zip(shapes.iterrows(), colors):

        feature_shape = folium.FeatureGroup(name=t_shape["shape_id"])

        shape = wkt.loads(t_shape["shape"])
        start_pt = wkt.loads(t_shape["start_pt"])
        end_pt = wkt.loads(t_shape["end_pt"])

        if shape.geom_type == 'LineString':
            line = [[y, x] for x, y in shape.coords]
        else:
            line = []
            for s in shape.geoms:
                for x, y in s.coords:
                    line.append([y, x])

        folium.PolyLine(line, color=t_color, line_weight=3).add_to(feature_shape)
        feature_shape.add_to(mapa_viagem)

        folium.Marker(
            location=[
                start_pt.y,
                start_pt.x
            ],
            tooltip=f"start_pt {t_shape['shape_id']}",
            icon=folium.Icon(color="green")
        ).add_to(feature_start_pt)

        folium.Circle(
            location=[
                start_pt.y,
                start_pt.x
            ],
            radius=500,
            fill=True,
            color="green",
            tooltip=f"start_pt {t_shape['shape_id']}",
        ).add_to(feature_start_pt)

        folium.Marker(
            location=[
                end_pt.y,
                end_pt.x
            ],
            tooltip=f"end_pt {t_shape['shape_id']}",
            icon=folium.Icon(color="red")
        ).add_to(feature_end_pt)

        folium.Circle(
            location=[
                end_pt.y,
                end_pt.x
            ],
            radius=500,
            fill=True,
            color="red",
            tooltip=f"end_pt {t_shape['shape_id']}",
        ).add_to(feature_end_pt)

    feature_start_pt.add_to(mapa_viagem)
    feature_end_pt.add_to(mapa_viagem)

    folium.LayerControl(collapsed=False).add_to(mapa_viagem)

    if(id_viagem is not None):
        print(f"id_viagem - {id_viagem}")
    else:
        print(f"{servico} - {sentido}")
        
    return f

## Análise exploratória


### Resumo da análise

Para identificar uma viagem circular, quebramos o trajeto em ida e volta no ponto médio.

#### Serviço: 010

*Problema:* posição do ponto médio cria um "laço" numa das metades do trajeto, onde sinais de GPS são descartados.

Exemplo: A viagem tem 25 min e só no trecho do laço foram descartados 9 min de sinais de GPS (~40%)

A partir de jun/23, mais de 75% das viagens tem 100% de GPS. Antes, valores variam entre altos e baixos. Até jun/23 o POD médio era de 82% (nível 2).

A partir de jun/23, foi para 118% (nível 1). Ponto intermediário foi alterado, removendo o laço do trajeto. Linha foi separada em ida e volta.

In [34]:
servicos = "010"

In [35]:
q = f"""
SELECT
  data,
  servico,
  viagens,
  km_planejada,
  km_apurada,
  perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = "{servicos}"
"""

df_sumario = bd.read_sql(q, from_file=True)

df_sumario["data"] = pd.to_datetime(df_sumario["data"])
df_sumario["mes"] = df_sumario["data"].apply(lambda x: x.replace(day=1))
df_sumario.to_csv(f"{paths['treated']}/sumario_servico_dia_historico.csv", index=False)

df_sumario.head()

Downloading: 100%|██████████| 422/422 [00:00<00:00, 1255.96rows/s]


,data,servico,viagens,km_planejada,km_apurada,perc_km_planejada,mes
0,2023-02-22,010,15,312.18,156.09,50.00,2023-02-01
1,2023-04-25,010,26,312.18,270.56,86.67,2023-04-01
2,2023-01-27,010,27,312.18,280.96,90.00,2023-01-01
3,2023-02-06,010,26,312.18,270.56,86.67,2023-02-01
4,2023-06-27,010,50,312.21,260.17,83.33,2023-06-01


In [36]:
df_sumario_treated = pd.read_csv(f"{paths['treated']}/sumario_servico_dia_historico.csv")
df_sumario_treated[["data", "mes"]] = df_sumario_treated[["data", "mes"]].apply(lambda x: pd.to_datetime(x))

# o valor de serviço deve ser string e precedido de 0
df_sumario_treated["servico"] = df_sumario_treated["servico"].astype(str)
df_sumario_treated["servico"] = df_sumario_treated["servico"].apply(lambda x: '0' + x)
df_sumario_treated

,data,servico,viagens,km_planejada,km_apurada,perc_km_planejada,mes
0,2023-02-22,010,15,312.180,156.090,50.00,2023-02-01
1,2023-04-25,010,26,312.180,270.560,86.67,2023-04-01
2,2023-01-27,010,27,312.180,280.960,90.00,2023-01-01
3,2023-02-06,010,26,312.180,270.560,86.67,2023-02-01
4,2023-06-27,010,50,312.210,260.170,83.33,2023-06-01
...,...,...,...,...,...,...,...
417,2022-11-18,010,26,250.067,270.894,108.33,2022-11-01
418,2023-01-08,010,10,145.680,104.060,71.43,2023-01-01
419,2023-06-11,010,31,77.070,161.010,208.91,2023-06-01
420,2023-05-14,010,8,145.680,83.250,57.15,2023-05-01


In [37]:
q = f"""
SELECT
  data,
  servico_informado AS servico,
  sentido,
  id_viagem,
  datetime_partida,
  datetime_chegada,
  perc_conformidade_shape,
  perc_conformidade_distancia,
  perc_conformidade_registros,
  perc_conformidade_tempo
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_completa`
WHERE
  servico_informado = "{servicos}"
"""

df_viagem_completa = bd.read_sql(q, from_file=True)
df_viagem_completa[["data", "datetime_partida"]] = df_viagem_completa[["data", "datetime_partida"]].apply(lambda x: pd.to_datetime(x))
df_viagem_completa["mes"] = df_viagem_completa["data"].apply(lambda x: x.replace(day=1))
df_viagem_completa = df_viagem_completa.sort_values(by="datetime_partida", ascending=False)
df_viagem_completa.to_csv(f"{paths['treated']}/viagem_completa.csv", index=False)

Downloading: 100%|██████████| 11105/11105 [00:02<00:00, 4681.70rows/s]


<!-- ### Hipóteses
- H1. Com critérios de conformidade, a linha nunca atinge o mínimo planejado válido;
- H2. Sem critérios de conformidade, a linha sempre atinge o mínimo planejado válido;
- H3. O limite de 80% de conformidade de shape faz com que a linha não atinja o mínimo planejado válido;
- H4. O limite de 50% de conformidade de registros de GPS  faz com que a linha não atinja o mínimo planejado válido. -->

In [91]:
df_viagem_completa_treated = pd.read_csv(f"{paths['treated']}/viagem_completa.csv")
df_viagem_completa_treated[["data", "mes", "datetime_partida", "datetime_chegada"]] = (df_viagem_completa_treated[["data", "mes", "datetime_partida", "datetime_chegada"]]
                                                                                       .apply(lambda x: pd.to_datetime(x)))


# Transformar a coluna "servico" em string
df_viagem_completa_treated["servico"] = df_viagem_completa_treated["servico"].astype(str)

# Adicionar um zero antes de cada valor na coluna "servico"
df_viagem_completa_treated["servico"] = df_viagem_completa_treated["servico"].apply(lambda x: '0' + x)

df_viagem_completa = dict()

for servico in servicos:
    df_viagem_completa[servico] = df_viagem_completa_treated[df_viagem_completa_treated["servico"] == '0' + servico].copy()
    
df_viagem_completa_treated

,data,servico,sentido,id_viagem,datetime_partida,datetime_chegada,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,perc_conformidade_tempo,mes
0,2023-08-31,010,I,A72149-010-I-01o6-20230831220015,2023-08-31 22:00:15,2023-08-31 22:11:56,100.0,103.57,100.00,0,2023-08-01
1,2023-08-31,010,I,A72108-010-I-01o6-20230831212833,2023-08-31 21:28:33,2023-08-31 21:41:33,100.0,100.16,100.00,0,2023-08-01
2,2023-08-31,010,V,A72149-010-V-53gi-20230831212222,2023-08-31 21:22:22,2023-08-31 21:37:09,100.0,96.64,100.00,0,2023-08-01
3,2023-08-31,010,V,A72108-010-V-53gi-20230831205452,2023-08-31 20:54:52,2023-08-31 21:10:11,100.0,94.75,100.00,0,2023-08-01
4,2023-08-31,010,I,A72149-010-I-01o6-20230831204359,2023-08-31 20:43:59,2023-08-31 20:56:01,100.0,99.55,100.00,0,2023-08-01
...,...,...,...,...,...,...,...,...,...,...,...
11100,2022-07-06,010,C,A72085-010-C-20220706092818,2022-07-06 09:28:18,2022-07-06 10:31:46,100.0,87.19,73.44,0,2022-07-01
11101,2022-07-06,010,C,A72048-010-C-20220706090523,2022-07-06 09:05:23,2022-07-06 10:15:58,100.0,87.76,67.61,0,2022-07-01
11102,2022-07-06,010,C,A72085-010-C-20220706082248,2022-07-06 08:22:48,2022-07-06 09:20:43,100.0,84.61,69.49,0,2022-07-01
11103,2022-07-06,010,C,A72048-010-C-20220706075241,2022-07-06 07:52:41,2022-07-06 08:51:51,100.0,84.19,55.00,0,2022-07-01


In [39]:
q = f"""
SELECT
  data,
  datetime_partida,
  servico_informado AS servico,
  perc_conformidade_shape,
  perc_conformidade_distancia,
  perc_conformidade_registros
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_conformidade`
WHERE
  servico_informado = "{servicos}"
"""

df_viagem_conformidade = bd.read_sql(q, from_file=True)
df_viagem_conformidade["data"] = df_viagem_conformidade["data"].apply(lambda x: pd.to_datetime(x))
df_viagem_conformidade["mes"] = df_viagem_conformidade["data"].apply(lambda x: x.replace(day=1))
df_viagem_conformidade = df_viagem_conformidade.sort_values(by="datetime_partida", ascending=False)
df_viagem_conformidade.to_csv(f"{paths['treated']}/viagem_conformidade.csv", index=False)

Downloading: 100%|██████████| 11582/11582 [00:01<00:00, 8214.18rows/s]


In [40]:
df_viagem_conformidade_treated = pd.read_csv(f"{paths['treated']}/viagem_conformidade.csv")
df_viagem_conformidade[["data", "mes", "datetime_partida"]] = (df_viagem_conformidade[["data", "mes", "datetime_partida"]]
                                                               .apply(lambda x: pd.to_datetime(x)))

df_viagem_conformidade = dict()

for servico in servicos:
    df_viagem_conformidade[servico] = (df_viagem_conformidade_treated[df_viagem_conformidade_treated["servico"] == servico]
                                       .copy())
    
df_viagem_conformidade_treated

,data,datetime_partida,servico,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,mes
0,2023-08-31,2023-08-31 22:00:15,10,100.0,103.57,100.00,2023-08-01
1,2023-08-31,2023-08-31 21:28:33,10,100.0,100.16,100.00,2023-08-01
2,2023-08-31,2023-08-31 21:22:22,10,100.0,96.64,100.00,2023-08-01
3,2023-08-31,2023-08-31 20:54:52,10,100.0,94.75,100.00,2023-08-01
4,2023-08-31,2023-08-31 20:43:59,10,100.0,99.55,100.00,2023-08-01
...,...,...,...,...,...,...,...
11577,2022-07-06,2022-07-06 09:28:18,10,100.0,87.19,73.44,2022-07-01
11578,2022-07-06,2022-07-06 09:05:23,10,100.0,87.76,67.61,2022-07-01
11579,2022-07-06,2022-07-06 08:22:48,10,100.0,84.61,69.49,2022-07-01
11580,2022-07-06,2022-07-06 07:52:41,10,100.0,84.19,55.00,2022-07-01


#### Distribuição do percentual dos parâmetros para validação das viagens

#### Análise exploratória de viagens

### Serviço: 010

#### Distribuição do percentual de operação diária (POD) por mês

In [49]:
servico = "010" 

In [42]:
fig = px.box(
    df_sumario_treated, 
    x="mes", 
    y="perc_km_planejada", 
    title=f"Distribuição do percentual de operação diária (POD) por mês - Serviço: {servico}",
    template="plotly_white",
    labels={
        "mes": "",
        "perc_km_planejada": ""
    },
    points="all"
)


fig.update_layout(
    yaxis_range=[-10, 250],
    yaxis_ticksuffix="%",
    # yaxis_tickprefix="R$ ",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=80, annotation_text="POD = 80%")

fig.show()

In [43]:
linha_corte = "2023-06-01"

In [44]:
df_sumario_treated[df_sumario_treated["data"] < linha_corte].describe()

# checar se os valores do notebook batem com os valores do notebook original

,viagens,km_planejada,km_apurada,perc_km_planejada
count,335.000000,335.000000,335.000000,335.000000
mean,19.146269,245.214045,199.337215,82.744836
std,7.764080,68.481012,80.868194,32.548501
min,0.000000,125.033000,0.000000,0.000000
25%,13.000000,166.490000,135.279000,70.000000
50%,22.000000,260.487000,229.218000,85.710000
75%,25.000000,312.180000,260.475000,100.000000
max,30.000000,312.180000,312.180000,186.660000


In [45]:
df_viagens_22 = df_sumario_treated[df_sumario_treated["mes"] <= linha_corte]

100 * len(df_viagens_22[df_viagens_22.perc_km_planejada >= 80]) / len(df_viagens_22)

65.47945205479452

* Junho/23 teve percentuais bema cima do geral
* Jul/22 e Fev/23 tem distribuição abaixo do esperado 

In [46]:
linha_corte = "2023-03-01"

In [53]:
df_sumario

,data,servico,viagens,km_planejada,km_apurada,perc_km_planejada,mes
0,2023-02-22,010,15,312.180,156.090,50.00,2023-02-01
1,2023-04-25,010,26,312.180,270.560,86.67,2023-04-01
2,2023-01-27,010,27,312.180,280.960,90.00,2023-01-01
3,2023-02-06,010,26,312.180,270.560,86.67,2023-02-01
4,2023-06-27,010,50,312.210,260.170,83.33,2023-06-01
...,...,...,...,...,...,...,...
417,2022-11-18,010,26,250.067,270.894,108.33,2022-11-01
418,2023-01-08,010,10,145.680,104.060,71.43,2023-01-01
419,2023-06-11,010,31,77.070,161.010,208.91,2023-06-01
420,2023-05-14,010,8,145.680,83.250,57.15,2023-05-01


In [63]:
df_sumario_22 = df_sumario_treated[df_sumario_treated['mes'] <= linha_corte]

100 * len(df_sumario_22[df_sumario_22.perc_km_planejada >= 80]) / len(df_sumario_22)

64.23357664233576

In [64]:
df_sumario_23 = df_sumario_treated[df_sumario_treated["mes"] > linha_corte]

100 * len(df_sumario_23[df_sumario_23.perc_km_planejada >= 80]) / len(df_sumario_23)

80.4054054054054

#### Distribuição do percentual dos parâmetros para validação das viagens

In [68]:
servico = "010"

In [69]:
y = "perc_conformidade_shape"
title = f"Distribuição do percentual de conformidade do shape por mês - Serviço: {servico}"
min = "80"

fig = px.box(
    df_viagem_conformidade_treated, 
    x="mes", 
    y=y, 
    title=title,
    template="plotly_white",
    labels={
        "mes": "",
        y: ""
    },
    points="all"
)


fig.update_layout(
    yaxis_range=[-10, 120],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=80, annotation_text=f"min = {min}%")

fig.show()

In [70]:
y = "perc_conformidade_registros"
title = f"Distribuição do percentual de conformidade de registros por mês - Serviço: {servico}"
min = 50

fig = px.box(
    df_viagem_conformidade_treated, 
    x="mes", 
    y=y, 
    title=title,
    template="plotly_white",
    labels={
        "mes": "",
        y: ""
    },
    points="all"
)


fig.update_layout(
    yaxis_range=[-10, 120],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=min, annotation_text=f"min = {str(min)}%")

fig.show()

In [71]:
df_viagem_conformidade_treated[df_viagem_conformidade_treated["data"] >= "2023-06-01"].describe()

,servico,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros
count,4692.0,4692.000000,4692.000000,4692.000000
mean,10.0,99.977121,97.047089,99.937882
std,0.0,0.571440,4.294555,1.300890
min,10.0,78.080000,53.200000,51.720000
25%,10.0,100.000000,95.070000,100.000000
50%,10.0,100.000000,96.720000,100.000000
75%,10.0,100.000000,98.310000,100.000000
max,10.0,100.000000,157.000000,100.000000


#### Análise exploratória de viagens

In [79]:
servico = "010"
data_ref_22 = "2022-12-29"
viagem_temp_22 = dict()

In [87]:
df_viagem_completa_treated[df_viagem_completa_treated["data"] == data_ref_22].head()

,data,servico,sentido,id_viagem,datetime_partida,datetime_chegada,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,perc_conformidade_tempo,mes
7454,2022-12-29,010,C,A72085-010-C-20221229204051,2022-12-29 20:40:51,2022-12-29 21:24:53,100.0,86.24,71.11,0,2022-12-01
7455,2022-12-29,010,C,A72037-010-C-20221229200528,2022-12-29 20:05:28,2022-12-29 20:49:07,100.0,87.86,71.11,0,2022-12-01
7456,2022-12-29,010,C,A72085-010-C-20221229194930,2022-12-29 19:49:30,2022-12-29 20:29:54,100.0,84.98,70.73,0,2022-12-01
7457,2022-12-29,010,C,A72037-010-C-20221229191753,2022-12-29 19:17:53,2022-12-29 20:01:28,100.0,86.32,68.89,0,2022-12-01
7458,2022-12-29,010,C,A72085-010-C-20221229185522,2022-12-29 18:55:22,2022-12-29 19:39:50,100.0,83.39,73.33,0,2022-12-01


In [103]:
servico = "010"
viagem_temp_22[servico] = df_viagem_completa_treated[df_viagem_completa_treated["data"] == data_ref_22].iloc[0].copy()

map_generate(servico, 
             viagem_temp_22[servico]["sentido"], 
             viagem_temp_22[servico]["data"].strftime("%Y-%m-%d"), 
             viagem_temp_22[servico]["id_viagem"], 
             viagem_temp_22[servico]["datetime_partida"], 
             viagem_temp_22[servico]["datetime_chegada"]
             )


        SELECT
          timestamp_gps,
          posicao_veiculo_geo,
          status_viagem
        FROM
          `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
        WHERE
          data = '2022-12-29'
          AND servico_informado = '010'
          AND id_viagem = 'A72085-010-C-20221229204051'
          AND timestamp_gps BETWEEN '2022-12-29 20:40:51' AND '2022-12-29 21:24:53'
          
        


Downloading: 100%|██████████| 91/91 [00:00<00:00, 255.61rows/s]



    SELECT
      *
    FROM
      `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
    WHERE
      servico = '010'
      AND data = '2022-12-29'
      AND sentido = 'C'
    


Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.45rows/s]

id_viagem - A72085-010-C-20221229204051


In [104]:
servico = "010"
data_ref_23 = "2023-02-01"
viagem_temp_23 = dict()

In [105]:
df_viagem_completa_treated[df_viagem_completa_treated["data"] == data_ref_23].head()

,data,servico,sentido,id_viagem,datetime_partida,datetime_chegada,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,perc_conformidade_tempo,mes
6839,2023-02-01,010,C,A72085-010-C-01o6-20230201210022,2023-02-01 21:00:22,2023-02-01 21:49:51,100.0,76.97,50.00,0,2023-02-01
6840,2023-02-01,010,C,A72037-010-C-01o6-20230201205055,2023-02-01 20:50:55,2023-02-01 21:32:48,100.0,77.69,60.47,0,2023-02-01
6841,2023-02-01,010,C,A72085-010-C-01o6-20230201200213,2023-02-01 20:02:13,2023-02-01 20:50:34,100.0,77.49,61.22,0,2023-02-01
6842,2023-02-01,010,C,A72037-010-C-01o6-20230201195021,2023-02-01 19:50:21,2023-02-01 20:42:38,100.0,74.71,54.72,0,2023-02-01
6843,2023-02-01,010,C,A72085-010-C-01o6-20230201190233,2023-02-01 19:02:33,2023-02-01 19:54:25,100.0,75.92,58.49,0,2023-02-01


In [106]:
viagem_temp_23[servico] = df_viagem_completa_treated[df_viagem_completa_treated["data"] == data_ref_23].iloc[0].copy()

map_generate(servico, 
             viagem_temp_23[servico]["sentido"], 
             viagem_temp_23[servico]["data"].strftime("%Y-%m-%d"), 
             viagem_temp_23[servico]["id_viagem"], 
             viagem_temp_23[servico]["datetime_partida"], 
             viagem_temp_23[servico]["datetime_chegada"]
             )


        SELECT
          timestamp_gps,
          posicao_veiculo_geo,
          status_viagem
        FROM
          `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
        WHERE
          data = '2023-02-01'
          AND servico_informado = '010'
          AND id_viagem = 'A72085-010-C-01o6-20230201210022'
          AND timestamp_gps BETWEEN '2023-02-01 21:00:22' AND '2023-02-01 21:49:51'
          
        


Downloading: 100%|██████████| 79/79 [00:00<00:00, 217.63rows/s]



    SELECT
      *
    FROM
      `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
    WHERE
      servico = '010'
      AND data = '2023-02-01'
      AND sentido = 'C'
    


Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.54rows/s]

id_viagem - A72085-010-C-01o6-20230201210022


In [107]:
servico = "010"
data_ref_23 = "2023-06-01"
viagem_temp_23 = dict()

In [109]:
df_viagem_completa_treated[df_viagem_completa_treated["data"] == data_ref_23].head()

,data,servico,sentido,id_viagem,datetime_partida,datetime_chegada,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,perc_conformidade_tempo,mes
4630,2023-06-01,010,V,A72085-010-V-01o6-20230601214204,2023-06-01 21:42:04,2023-06-01 21:51:13,100.0,95.29,100.0,0,2023-06-01
4631,2023-06-01,010,I,A72085-010-I-53gi-20230601211227,2023-06-01 21:12:27,2023-06-01 21:32:02,100.0,93.71,100.0,0,2023-06-01
4632,2023-06-01,010,V,A72181-010-V-01o6-20230601210323,2023-06-01 21:03:23,2023-06-01 21:15:11,100.0,98.78,100.0,0,2023-06-01
4633,2023-06-01,010,I,A72181-010-I-53gi-20230601204326,2023-06-01 20:43:26,2023-06-01 20:58:49,100.0,97.33,100.0,0,2023-06-01
4634,2023-06-01,010,V,A72085-010-V-01o6-20230601203712,2023-06-01 20:37:12,2023-06-01 20:52:04,100.0,97.61,100.0,0,2023-06-01


In [110]:
viagem_temp_23[servico] = df_viagem_completa_treated[df_viagem_completa_treated["data"] == data_ref_23].iloc[0].copy()

map_generate(servico, 
             None, 
             viagem_temp_23[servico]["data"].strftime("%Y-%m-%d"), 
             None, 
             "2023-06-01 21:12:27", 
             viagem_temp_23[servico]["datetime_chegada"]
             )


        SELECT
          timestamp_gps,
          posicao_veiculo_geo,
          status_viagem
        FROM
          `rj-smtr.projeto_subsidio_sppo.registros_status_viagem`
        WHERE
          data = '2023-06-01'
          AND servico_informado = '010'
          
          AND timestamp_gps BETWEEN '2023-06-01 21:12:27' AND '2023-06-01 21:51:13'
          
        


Downloading: 100%|██████████| 530/530 [00:00<00:00, 1540.74rows/s]



    SELECT
      *
    FROM
      `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
    WHERE
      servico = '010'
      AND data = '2023-06-01'
      
    


Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.85rows/s]

010 - 


#### Análise exploratória das informações enviadas pela operadora

In [111]:
servico = "010"
data_ref_23 = "2023-01-02"
viagem_temp_23 = dict()

In [112]:
file_path = f"{paths['raw']}/20230818_viagens_010_1q_jan_2023_treated.xls"
df_viagens_operador = dict()
servico = "010"

In [115]:
df = pd.read_excel(file_path, sheet_name="02-01")
df = df[["Data", "Linha", "Veículo", "Saída", "Cheg.", "Saída.1", "Cheg..1"]]
df = df.rename(columns={"Data": "data",
                        "Linha": "servico",
                        "Veículo": "id_veiculo",
                        "Saída": "time_partida",
                        "Cheg.": "time_chegada_intermediario",
                        "Saída.1": "time_partida_intermediario",
                        "Cheg..1": "time_chegada"})

In [116]:
df_temp = dict()
df_temp["V"] = df[["data", "servico", "id_veiculo", "time_partida", "time_chegada_intermediario"]].rename(columns={"time_chegada_intermediario": "time_chegada"})
df_temp["V"]["sentido"] = "V"
df_temp["I"] = df[["data", "servico", "id_veiculo", "time_partida_intermediario", "time_chegada"]].rename(columns={"time_partida_intermediario": "time_partida"})
df_temp["I"]["sentido"] = "I"

df = pd.concat([df_temp["I"], df_temp["V"]])
df = df[~(df["time_partida"].isna() & df["time_chegada"].isna())]
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   data          707 non-null    datetime64[ns]
 1   servico       707 non-null    int64         
 2   id_veiculo    707 non-null    object        
 3   time_partida  707 non-null    object        
 4   time_chegada  707 non-null    object        
 5   sentido       707 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 33.3+ KB


In [117]:
df[['time_partida',
     'time_chegada']] = df[['time_partida',
                             'time_chegada']].astype(str)

df["servico"] = df["servico"].astype(str)
df["data"] = df["data"].astype(str)
df["servico"] = servico

In [118]:
df["datetime_partida"] = pd.to_datetime(df["data"]+ " " + df["time_partida"], errors="coerce")
df["datetime_chegada"] = pd.to_datetime(df["data"]+ " " + df["time_chegada"], errors="coerce")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   data              707 non-null    object        
 1   servico           707 non-null    object        
 2   id_veiculo        707 non-null    object        
 3   time_partida      707 non-null    object        
 4   time_chegada      707 non-null    object        
 5   sentido           707 non-null    object        
 6   datetime_partida  707 non-null    datetime64[ns]
 7   datetime_chegada  707 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(6)
memory usage: 44.3+ KB


In [119]:
df.loc[(df["datetime_partida"] != (df["data"]+ " " + df["time_partida"])), "datetime_partida"] = (pd.to_datetime(
    df[df["datetime_partida"] != (df["data"]+ " " + df["time_partida"])]["data"]+ " " + 
    df[df["datetime_partida"] != (df["data"]+ " " + df["time_partida"])]["time_partida"]))

In [120]:
df.loc[(df["datetime_chegada"] != (df["data"]+ " " + df["time_chegada"])), "datetime_chegada"] = (pd.to_datetime(
    df[df["datetime_chegada"] != (df["data"]+ " " + df["time_chegada"])]["data"]+ " " + 
    df[df["datetime_chegada"] != (df["data"]+ " " + df["time_chegada"])]["time_chegada"]))

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   data              707 non-null    object        
 1   servico           707 non-null    object        
 2   id_veiculo        707 non-null    object        
 3   time_partida      707 non-null    object        
 4   time_chegada      707 non-null    object        
 5   sentido           707 non-null    object        
 6   datetime_partida  707 non-null    datetime64[ns]
 7   datetime_chegada  707 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(6)
memory usage: 44.3+ KB


In [122]:
df = df[["data", "servico", "sentido", "id_veiculo", "datetime_partida", "datetime_chegada"]]
df = df.sort_values("datetime_partida")
df["data"] = pd.to_datetime(df["data"])
df.to_csv(f"{paths['treated']}/viagens_010_1q_jan_2023.csv", index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707 entries, 0 to 363
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   data              707 non-null    datetime64[ns]
 1   servico           707 non-null    object        
 2   sentido           707 non-null    object        
 3   id_veiculo        707 non-null    object        
 4   datetime_partida  707 non-null    datetime64[ns]
 5   datetime_chegada  707 non-null    datetime64[ns]
dtypes: datetime64[ns](3), object(3)
memory usage: 38.7+ KB


In [123]:
df_viagens_operador[servico] = pd.read_csv(f"{paths['treated']}/viagens_010_1q_jan_2023.csv")
df_viagens_operador[servico]["servico"] = servico
df_viagens_operador[servico].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   data              707 non-null    object
 1   servico           707 non-null    object
 2   sentido           707 non-null    object
 3   id_veiculo        707 non-null    object
 4   datetime_partida  707 non-null    object
 5   datetime_chegada  707 non-null    object
dtypes: object(6)
memory usage: 33.3+ KB


In [124]:
df_viagens_operador[servico].head()

,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada
0,2023-01-02,010,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00
1,2023-01-02,010,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00
2,2023-01-02,010,I,A72085,2023-01-02 06:33:00,2023-01-02 06:52:00
3,2023-01-02,010,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00
4,2023-01-02,010,V,A72085,2023-01-02 07:03:00,2023-01-02 07:29:00


In [126]:
df_viagem_completa_treated[df_viagem_completa_treated["data"] == data_ref_23].sort_values(["datetime_partida"]).head()

,data,servico,sentido,id_viagem,datetime_partida,datetime_chegada,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,perc_conformidade_tempo,mes
7429,2023-01-02,010,C,A72085-010-C-01o6-20230102070549,2023-01-02 07:05:49,2023-01-02 07:50:11,100.0,75.75,60.87,0,2023-01-01
7428,2023-01-02,010,C,A72037-010-C-01o6-20230102073147,2023-01-02 07:31:47,2023-01-02 08:22:04,100.0,74.14,57.69,0,2023-01-01
7427,2023-01-02,010,C,A72085-010-C-01o6-20230102080645,2023-01-02 08:06:45,2023-01-02 08:52:47,100.0,73.72,70.21,0,2023-01-01
7426,2023-01-02,010,C,A72037-010-C-01o6-20230102083445,2023-01-02 08:34:45,2023-01-02 09:29:14,100.0,77.87,51.79,0,2023-01-01
7425,2023-01-02,010,C,A72037-010-C-01o6-20230102093754,2023-01-02 09:37:54,2023-01-02 10:24:38,100.0,76.24,58.33,0,2023-01-01


In [127]:
servico = "010"
data_ref_23 = "2023-01-02"
viagem_temp_23 = dict()

In [129]:
viagem_temp_23[servico] = df_viagem_completa_treated[df_viagem_completa_treated["data"] == data_ref_23].iloc[0].copy()

map_generate(servico, 
             viagem_temp_23[servico]["sentido"], 
             viagem_temp_23[servico]["data"].strftime("%Y-%m-%d"), 
             viagem_temp_23[servico]["id_viagem"], 
             "2023-01-02 06:29:00", 
             "2023-01-02 07:16:00",
             table_id_gps="gps_sppo",
             id_veiculo="A72037"
             )


        SELECT
          timestamp_gps,
          'middle' AS status_viagem,
          ST_GEOGPOINT(longitude, latitude) AS posicao_veiculo_geo
        FROM
          `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
        WHERE
          data = '2023-01-02'
          AND servico = '010'
          AND timestamp_gps BETWEEN '2023-01-02 06:29:00' AND '2023-01-02 07:16:00'
          AND id_veiculo = 'A72037'
        


Downloading: 100%|██████████| 121/121 [00:00<00:00, 361.19rows/s]



    SELECT
      *
    FROM
      `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
    WHERE
      servico = '010'
      AND data = '2023-01-02'
      AND sentido = 'C'
    


Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.13rows/s]


id_viagem - A72037-010-C-01o6-20230102192824


### Alteração no método de avaliação de status dos pontos GPS

Com base na análise exploratória, o indicador de conformidade de registros foi identificado como fator relevante que tem impactado na identificação de viagens das linhas. A mediana encontrar-se abaixo do limite do indicador (50%) apresenta-se como ponto relevante de análise. Visando tentar aprimorar esse indicador, desenvolveremos um novo método de avaliação de status dos pontos GPS.

Nesse método, a identificação do status dos pontos GPS será realizada através do ponto no shape mais próximo do GPS, identificando a distância percorrida no shape, e não apenas através do raio de 500 m no método atual.

As alterações foram incorporadas no DBT e serão testadas. Neste momento, utilizaremos o dataset `rj-smtr-dev.20230815_projeto_subsidio_sppo_circulares`, também fruto da execução do model `projeto_subsidio_sppo` no DBT. As alterações foram consolidadas no seguinte PR: https://github.com/prefeitura-rio/queries-rj-smtr/pull/52

Importante ressaltar que houve alteração no método de identificação de itinerários circulares a partir de `2023-01-01`. Dessa forma, deve-se levar em consideração quando do reprocessamento de datas anteriores.

#### Resumo da análise da proposta

**Serviço 010:** Proposta não solucionou. Testar troca do ponto intermediário.

In [133]:
servicos = "010"

In [135]:
q = f"""
SELECT
  data,
  servico_informado AS servico,
  sentido,
  id_viagem,
  datetime_partida,
  datetime_chegada,
  perc_conformidade_shape,
  perc_conformidade_distancia,
  perc_conformidade_registros,
  perc_conformidade_tempo
FROM
  `rj-smtr-dev.20230815_projeto_subsidio_sppo_circulares.viagem_completa`
WHERE
  servico_informado = '{servicos}'
"""

df_viagem_completa_dev = bd.read_sql(q, from_file=True)
df_viagem_completa_dev[["data", "datetime_partida"]] = df_viagem_completa_dev[["data", "datetime_partida"]].apply(lambda x: pd.to_datetime(x))
df_viagem_completa_dev["mes"] = df_viagem_completa_dev["data"].apply(lambda x: x.replace(day=1))
df_viagem_completa_dev = df_viagem_completa_dev.sort_values(by="datetime_partida", ascending=False)
df_viagem_completa_dev.to_csv(f"{paths['treated']}/viagem_completa_dev.csv", index=False)

Downloading: 100%|██████████| 130/130 [00:00<00:00, 370.37rows/s]


#### Serviço: 010

In [136]:
servico = "010"
data_ref_22 = "2022-12-29"

##### Análise de Indicadores

In [138]:
len(df_viagem_completa_dev[df_viagem_completa_dev["data"] == data_ref_22])

26

In [139]:
len(df_viagem_completa_dev[df_viagem_completa_dev["data"] == data_ref_22])

26

Não foi identificada diferença na quantidade de viagens identificadas com os métodos diferentes na amostra selecionada.

##### Análise Exploratória das Viagens

In [141]:
viagem_temp_22[servico] = df_viagem_completa_dev[df_viagem_completa_dev["data"] == data_ref_22].iloc[0].copy()

map_generate(servico, 
             viagem_temp_22[servico]["sentido"], 
             viagem_temp_22[servico]["data"].strftime("%Y-%m-%d"), 
             viagem_temp_22[servico]["id_viagem"], 
             viagem_temp_22[servico]["datetime_partida"], 
             viagem_temp_22[servico]["datetime_chegada"],
             mode="dev"
             )


        SELECT
          timestamp_gps,
          posicao_veiculo_geo,
          status_viagem
        FROM
          `rj-smtr-dev.20230815_projeto_subsidio_sppo_circulares.registros_status_viagem`
        WHERE
          data = '2022-12-29'
          AND servico_informado = '010'
          AND id_viagem = 'A72085-010-C-20221229204011'
          AND timestamp_gps BETWEEN '2022-12-29 20:40:11' AND '2022-12-29 21:26:13'
          
        


Downloading: 100%|██████████| 108/108 [00:00<00:00, 340.68rows/s]



    SELECT
      *
    FROM
      `rj-smtr-dev.20230815_projeto_subsidio_sppo_circulares.viagem_planejada`
    WHERE
      servico = '010'
      AND data = '2022-12-29'
      AND sentido = 'C'
    


Downloading: 100%|██████████| 2/2 [00:00<00:00,  4.76rows/s]

id_viagem - A72085-010-C-20221229204011


### Alteração de ponto intermediário

Com base na análise exploratória, algumas linhas aumentaram o POD através da alteração de circular para ida/volta. Nesse sentido, será verificado se essa alteração do planejado impactaria nas viagens de datas anteriores.

Nesse método, será utilizado o planejado de `2023-06-01` para a data de `2023-01-02`.

As alterações foram incorporadas na tabela `subsidio_data_versao_efetiva` no DBT e serão testadas. Neste momento, utilizaremos o dataset `rj-smtr-dev.20230818_projeto_subsidio_sppo_010`, também fruto da execução do model `projeto_subsidio_sppo` no DBT. 

#### Resumo da análise da proposta

<!-- **Serviço 010:** Proposta não solucionou. Testar troca do ponto intermediário.

**Serviço 663:** Sem problema metodológico. Validar viagens com amostra do consórcio.

**Serviço LECD50:** Analisar problema e testar solução.

**Serviço 685:** Analisar problema e testar solução. -->

In [142]:
servico = "010"

In [143]:
q = f"""
SELECT
  data,
  servico_informado AS servico,
  sentido,
  id_viagem,
  id_veiculo,
  datetime_partida,
  datetime_chegada,
  distancia_planejada,
  perc_conformidade_shape,
  perc_conformidade_distancia,
  perc_conformidade_registros,
  perc_conformidade_tempo
FROM
  `rj-smtr-dev.20230818_projeto_subsidio_sppo_010.viagem_completa`
WHERE
  servico_informado = '{servico}'
"""

df_viagem_completa_dev = bd.read_sql(q, from_file=True)
df_viagem_completa_dev[["data", "datetime_partida"]] = df_viagem_completa_dev[["data", "datetime_partida"]].apply(lambda x: pd.to_datetime(x))
df_viagem_completa_dev["mes"] = df_viagem_completa_dev["data"].apply(lambda x: x.replace(day=1))
df_viagem_completa_dev = df_viagem_completa_dev.sort_values(by="datetime_partida")
df_viagem_completa_dev.to_csv(f"{paths['treated']}/viagem_completa_dev_010.csv", index=False)

Downloading: 100%|██████████| 734/734 [00:00<00:00, 1821.34rows/s]


In [144]:
df_viagem_completa_dev_treated = pd.read_csv(f"{paths['treated']}/viagem_completa_dev_010.csv")
df_viagem_completa_dev_treated["servico"] = servico
df_viagem_completa_dev_treated[["data", "mes", "datetime_partida", "datetime_chegada"]] = (df_viagem_completa_dev_treated[["data", "mes", "datetime_partida", "datetime_chegada"]]
                                                                                           .apply(lambda x: pd.to_datetime(x)))
df_viagem_completa_dev = dict()

df_viagem_completa_dev[servico] = (df_viagem_completa_dev_treated[df_viagem_completa_dev_treated["servico"] == servico]
                                   .copy())

#### Serviço: 010

In [145]:
servico = "010"
data_ref_23 = "2023-01-02"

##### Análise de Indicadores

In [146]:
df_viagens_operador = dict()
df_viagens_operador[servico] = pd.read_csv(f"{paths['treated']}/viagens_010_1q_jan_2023.csv")
df_viagens_operador[servico]["servico"] = servico
df_viagens_operador[servico]["data"] = pd.to_datetime(df_viagens_operador[servico]["data"])
df_viagens_operador[servico].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   data              707 non-null    datetime64[ns]
 1   servico           707 non-null    object        
 2   sentido           707 non-null    object        
 3   id_veiculo        707 non-null    object        
 4   datetime_partida  707 non-null    object        
 5   datetime_chegada  707 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 33.3+ KB


In [147]:
df_viagem_completa_dev[servico][df_viagem_completa_dev[servico]["data"] == data_ref_23].head()

,data,servico,sentido,id_viagem,id_veiculo,datetime_partida,datetime_chegada,distancia_planejada,perc_conformidade_shape,perc_conformidade_distancia,perc_conformidade_registros,perc_conformidade_tempo,mes
0,2023-01-02,010,I,A72037-010-I-53gi-20230102060757,A72037,2023-01-02 06:07:57,2023-01-02 06:19:58,5.505,100.0,94.24,100.0,0,2023-01-01
1,2023-01-02,010,V,A72037-010-V-01o6-20230102063200,A72037,2023-01-02 06:32:00,2023-01-02 06:43:21,4.902,100.0,95.12,100.0,0,2023-01-01
2,2023-01-02,010,I,A72085-010-I-53gi-20230102063511,A72085,2023-01-02 06:35:11,2023-01-02 06:47:51,5.505,100.0,92.39,100.0,0,2023-01-01
3,2023-01-02,010,I,A72037-010-I-53gi-20230102070004,A72037,2023-01-02 07:00:04,2023-01-02 07:13:44,5.505,100.0,92.92,100.0,0,2023-01-01
4,2023-01-02,010,V,A72085-010-V-01o6-20230102070529,A72085,2023-01-02 07:05:29,2023-01-02 07:21:50,4.902,100.0,94.96,100.0,0,2023-01-01


In [148]:
len(df_viagens_operador[servico])

707

In [149]:
len(df_viagem_completa_dev[servico][((df_viagem_completa_dev[servico]["data"] >= "2023-01-02") &
                                     (df_viagem_completa_dev[servico]["data"] <= "2023-01-15"))]) # Viagens de ida e volta

734

In [152]:
len(df_viagem_completa[servico][((df_viagem_completa[servico]["data"] >= "2023-01-02") & 
                                 (df_viagem_completa[servico]["data"] <= "2023-01-15"))])

KeyError: 'data'

In [153]:
df_sumario_solucao = dict()
df_viagens_solucao = dict()

In [154]:
q = """
SELECT
  DISTINCT data,
  servico,
  sentido,
  distancia_planejada AS extensao
FROM
  rj-smtr-dev.20230818_projeto_subsidio_sppo_010.viagem_planejada
WHERE
  DATA BETWEEN "2023-01-02"
  AND "2023-01-15"
  AND servico = "010"
"""

df_viagens_solucao[servico] = bd.read_sql(q, from_file=True)
df_viagens_solucao[servico]["data"] = pd.to_datetime(df_viagens_solucao[servico]["data"])
df_viagens_solucao[servico].head()

Downloading: 100%|██████████| 28/28 [00:00<00:00, 86.96rows/s]


,data,servico,sentido,extensao
0,2023-01-13,010,V,4.902
1,2023-01-13,010,I,5.505
2,2023-01-02,010,V,4.902
3,2023-01-02,010,I,5.505
4,2023-01-15,010,I,5.505


In [155]:
df_viagens_solucao[servico].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   data      28 non-null     datetime64[ns]
 1   servico   28 non-null     object        
 2   sentido   28 non-null     object        
 3   extensao  28 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.0+ KB


In [156]:
df_agg_temp = (df_viagens_operador[servico]
               .groupby(["data", "servico", "sentido"])
               .count()["id_veiculo"]
               .reset_index()
               .rename(columns={"id_veiculo": "q_viagens"}))

df_agg_temp["data"] = pd.to_datetime(df_agg_temp["data"])
df_agg_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   data       28 non-null     datetime64[ns]
 1   servico    28 non-null     object        
 2   sentido    28 non-null     object        
 3   q_viagens  28 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.0+ KB


In [157]:
df_agg_temp = df_agg_temp.merge(df_viagens_solucao[servico], on=["data", "servico", "sentido"], how="left")
df_agg_temp["km_apurada_gabarito"] = df_agg_temp["q_viagens"]*df_agg_temp["extensao"]
df_agg_temp.head()

,data,servico,sentido,q_viagens,extensao,km_apurada_gabarito
0,2023-01-02,010,I,29,5.505,159.645
1,2023-01-02,010,V,27,4.902,132.354
2,2023-01-03,010,I,28,5.505,154.140
3,2023-01-03,010,V,26,4.902,127.452
4,2023-01-04,010,I,30,5.505,165.150


In [158]:
df_agg_temp = (df_agg_temp
               .groupby(["data", "servico"])
               .sum()["km_apurada_gabarito"]
               .reset_index())
df_agg_temp.head()

,data,servico,km_apurada_gabarito
0,2023-01-02,010,291.999
1,2023-01-03,010,281.592
2,2023-01-04,010,302.406
3,2023-01-05,010,286.494
4,2023-01-06,010,291.396


In [159]:
q = """
WITH
  planejado AS (
  SELECT
    DISTINCT DATA,
    servico,
    sentido,
    distancia_planejada AS extensao
  FROM
    rj-smtr.projeto_subsidio_sppo.viagem_planejada
  WHERE
    DATA BETWEEN "2023-01-02"
    AND "2023-01-15"
    AND servico = "010" ),
  sumario AS (
  SELECT
    DATA,
    servico,
    km_planejada,
    perc_km_planejada
  FROM
    rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico
  WHERE
    DATA BETWEEN "2023-01-02"
    AND "2023-01-15"
    AND servico = "010" ),
  solucao AS (
  SELECT
    DATA,
    servico_realizado AS servico,
    SUM(distancia_planejada) AS km_solucao
  FROM
    rj-smtr-dev.20230818_projeto_subsidio_sppo_010.viagem_completa
  WHERE
    DATA BETWEEN "2023-01-02"
    AND "2023-01-15"
    AND servico_realizado = "010"
  GROUP BY
    1,
    2 )
SELECT
  p.data,
  p.servico,
  s.km_planejada,
  s.perc_km_planejada AS pod_apuracao,
  ROUND(100 * km_solucao / s.km_planejada, 2) AS pod_solucao
FROM
  planejado p
LEFT JOIN
  sumario s
USING
  (DATA,
    servico)
LEFT JOIN
  solucao r
USING
  (DATA,
    servico)
"""

df_sumario_solucao[servico] = bd.read_sql(q, from_file=True)
df_sumario_solucao[servico]["data"] = pd.to_datetime(df_sumario_solucao[servico]["data"])
df_sumario_solucao[servico].head()

Downloading: 100%|██████████| 14/14 [00:00<00:00, 32.71rows/s]


,data,servico,km_planejada,pod_apuracao,pod_solucao
0,2023-01-14,010,166.50,62.50,100.01
1,2023-01-10,010,312.18,63.33,90.01
2,2023-01-06,010,312.18,70.00,100.01
3,2023-01-03,010,312.18,63.33,93.34
4,2023-01-11,010,312.18,76.67,100.01


In [160]:
df_sumario_solucao[servico] = df_sumario_solucao[servico].merge(df_agg_temp, on=["data", "servico"], how="left")
df_sumario_solucao[servico]["pod_gabarito"] = round((df_sumario_solucao[servico]["km_apurada_gabarito"]/
                                                     df_sumario_solucao[servico]["km_planejada"])*100,2)
df_sumario_solucao[servico].head()

,data,servico,km_planejada,pod_apuracao,pod_solucao,km_apurada_gabarito,pod_gabarito
0,2023-01-14,010,166.50,62.50,100.01,151.203,90.81
1,2023-01-10,010,312.18,63.33,90.01,271.185,86.87
2,2023-01-06,010,312.18,70.00,100.01,291.396,93.34
3,2023-01-03,010,312.18,63.33,93.34,281.592,90.20
4,2023-01-11,010,312.18,76.67,100.01,302.406,96.87


In [161]:
df_sumario_solucao[servico] = df_sumario_solucao[servico][df_sumario_solucao[servico].columns.difference(["km_apurada_gabarito"])]
df_sumario_solucao[servico] = df_sumario_solucao[servico].sort_values(["data"]).reset_index(drop=True)
df_sumario_solucao[servico].head()

,data,km_planejada,pod_apuracao,pod_gabarito,pod_solucao,servico
0,2023-01-02,312.18,76.67,93.54,96.68,010
1,2023-01-03,312.18,63.33,90.20,93.34,010
2,2023-01-04,312.18,70.00,96.87,91.58,010
3,2023-01-05,312.18,73.33,91.77,98.44,010
4,2023-01-06,312.18,70.00,93.34,100.01,010


In [439]:
df_sumario_solucao[servico].to_csv(f"{paths['treated']}/sumario_solucao_010.csv", index=False)

In [169]:
df_viagens_operador_ext = dict()

In [170]:
df_viagens_operador_ext[servico] = (df_viagens_operador[servico].merge(df_viagens_solucao[servico], 
                                                                       on=["data", "servico", "sentido"], 
                                                                       how="left"))

df_viagens_operador_ext[servico].head()

,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,extensao
0,2023-01-02,010,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,5.505
1,2023-01-02,010,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00,4.902
2,2023-01-02,010,I,A72085,2023-01-02 06:33:00,2023-01-02 06:52:00,5.505
3,2023-01-02,010,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00,5.505
4,2023-01-02,010,V,A72085,2023-01-02 07:03:00,2023-01-02 07:29:00,4.902


In [171]:
df_viagens_operador_ext[servico].to_csv(f"{paths['treated']}/viagens_010_1q_jan_2023_ext.csv", index=False)

In [172]:
df_viagens_operador_ext[servico] = pd.read_csv(f"{paths['treated']}/viagens_010_1q_jan_2023_ext.csv")
df_viagens_operador_ext[servico][["data", "datetime_partida", "datetime_chegada"]] = (df_viagens_operador_ext[servico][["data", "datetime_partida", "datetime_chegada"]]
                                                                                      .apply(lambda x: pd.to_datetime(x)))
df_viagens_operador_ext[servico]["servico"] = servico
df_viagens_operador_ext[servico].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   data              707 non-null    datetime64[ns]
 1   servico           707 non-null    object        
 2   sentido           707 non-null    object        
 3   id_veiculo        707 non-null    object        
 4   datetime_partida  707 non-null    datetime64[ns]
 5   datetime_chegada  707 non-null    datetime64[ns]
 6   extensao          707 non-null    float64       
dtypes: datetime64[ns](3), float64(1), object(3)
memory usage: 38.8+ KB


In [173]:
df_viagem_completa_dev[servico].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 734 entries, 0 to 733
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   data                         734 non-null    datetime64[ns]
 1   servico                      734 non-null    object        
 2   sentido                      734 non-null    object        
 3   id_viagem                    734 non-null    object        
 4   id_veiculo                   734 non-null    object        
 5   datetime_partida             734 non-null    datetime64[ns]
 6   datetime_chegada             734 non-null    datetime64[ns]
 7   distancia_planejada          734 non-null    float64       
 8   perc_conformidade_shape      734 non-null    float64       
 9   perc_conformidade_distancia  734 non-null    float64       
 10  perc_conformidade_registros  734 non-null    float64       
 11  perc_conformidade_tempo      734 non-null    

In [174]:
len(df_viagens_operador_ext[servico][df_viagens_operador_ext[servico]["data"] == "2023-01-02"])

56

In [175]:
df_viagens_operador_ext[servico].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 707 entries, 0 to 706
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   data              707 non-null    datetime64[ns]
 1   servico           707 non-null    object        
 2   sentido           707 non-null    object        
 3   id_veiculo        707 non-null    object        
 4   datetime_partida  707 non-null    datetime64[ns]
 5   datetime_chegada  707 non-null    datetime64[ns]
 6   extensao          707 non-null    float64       
dtypes: datetime64[ns](3), float64(1), object(3)
memory usage: 38.8+ KB


In [176]:
len(df_viagem_completa_dev[servico][df_viagem_completa_dev[servico]["data"] == "2023-01-02"])

58

In [177]:
df_merged = pd.merge_asof((df_viagem_completa_dev[servico][['data', 'servico', 'sentido', 'id_veiculo', 'datetime_partida', 'datetime_chegada', 'distancia_planejada']]
                           .sort_values("datetime_partida")
                           .rename(columns={"distancia_planejada": "extensao"})), 
                          (df_viagens_operador_ext[servico][['data', 'servico', 'sentido', 'id_veiculo', 'datetime_partida', 'datetime_chegada']]
                           .sort_values("datetime_partida")
                           .rename(columns={"datetime_partida": "datetime_partida_gabarito"})),
                          left_on="datetime_partida",
                          right_on="datetime_partida_gabarito",
                          by=["data", "id_veiculo", "sentido", "servico"],
                          suffixes=("", "_gabarito"),
                          direction="backward",
                          tolerance=timedelta(minutes=30),
                         )
df_merged.head()

,data,servico,sentido,id_veiculo,datetime_partida,datetime_chegada,extensao,datetime_partida_gabarito,datetime_chegada_gabarito
0,2023-01-02,010,I,A72037,2023-01-02 06:07:57,2023-01-02 06:19:58,5.505,2023-01-02 06:07:00,2023-01-02 06:22:00
1,2023-01-02,010,V,A72037,2023-01-02 06:32:00,2023-01-02 06:43:21,4.902,2023-01-02 06:29:00,2023-01-02 06:56:00
2,2023-01-02,010,I,A72085,2023-01-02 06:35:11,2023-01-02 06:47:51,5.505,2023-01-02 06:33:00,2023-01-02 06:52:00
3,2023-01-02,010,I,A72037,2023-01-02 07:00:04,2023-01-02 07:13:44,5.505,2023-01-02 06:58:00,2023-01-02 07:16:00
4,2023-01-02,010,V,A72085,2023-01-02 07:05:29,2023-01-02 07:21:50,4.902,2023-01-02 07:03:00,2023-01-02 07:29:00


In [178]:
df_merged.to_csv(f"{paths['treated']}/viagens_010_solucao_gabarito_merged.csv", index=False)

In [179]:
df_merged = pd.merge_asof(df_viagens_operador_ext[servico].sort_values("datetime_partida"),
                          df_viagem_completa_dev[servico][['data', 'servico', 'sentido', 'id_viagem', 'id_veiculo', 'datetime_partida', 'datetime_chegada']].sort_values("datetime_partida"), 
                          on=["datetime_partida"],
                          by=["data", "id_veiculo", "sentido"],
                          suffixes=("_gabarito","_solucao"),
                          direction="forward",
                          tolerance=timedelta(minutes=30)
                         )
df_merged.head()

,data,servico_gabarito,sentido,id_veiculo,datetime_partida,datetime_chegada_gabarito,extensao,servico_solucao,id_viagem,datetime_chegada_solucao
0,2023-01-02,010,I,A72037,2023-01-02 06:07:00,2023-01-02 06:22:00,5.505,010,A72037-010-I-53gi-20230102060757,2023-01-02 06:19:58
1,2023-01-02,010,V,A72037,2023-01-02 06:29:00,2023-01-02 06:56:00,4.902,010,A72037-010-V-01o6-20230102063200,2023-01-02 06:43:21
2,2023-01-02,010,I,A72085,2023-01-02 06:33:00,2023-01-02 06:52:00,5.505,010,A72085-010-I-53gi-20230102063511,2023-01-02 06:47:51
3,2023-01-02,010,I,A72037,2023-01-02 06:58:00,2023-01-02 07:16:00,5.505,010,A72037-010-I-53gi-20230102070004,2023-01-02 07:13:44
4,2023-01-02,010,V,A72085,2023-01-02 07:03:00,2023-01-02 07:29:00,4.902,010,A72085-010-V-01o6-20230102070529,2023-01-02 07:21:50


In [180]:
df_merged.to_csv(f"{paths['treated']}/viagens_010_gabarito_solucao_merged.csv", index=False)

In [181]:
df_merged_agg = df_merged.groupby("data")[["datetime_chegada_gabarito", "datetime_chegada_solucao"]].count().reset_index().rename(columns={"datetime_chegada_gabarito": "q_viagens_gabarito", "datetime_chegada_solucao": "q_viagens_solucao"})
df_merged_agg

,data,q_viagens_gabarito,q_viagens_solucao
0,2023-01-02,56,56
1,2023-01-03,54,53
2,2023-01-04,58,53
3,2023-01-05,55,55
4,2023-01-06,56,56
5,2023-01-07,54,54
6,2023-01-08,31,31
7,2023-01-09,58,58
8,2023-01-10,52,52
9,2023-01-11,58,58


In [182]:
df_viagem_completa_dev_agg = df_viagem_completa_dev[servico].groupby("data")[["datetime_chegada"]].count().reset_index().rename(columns={"datetime_chegada": "q_viagens_solucao"})
df_viagem_completa_dev_agg

,data,q_viagens_solucao
0,2023-01-02,58
1,2023-01-03,56
2,2023-01-04,55
3,2023-01-05,59
4,2023-01-06,60
5,2023-01-07,56
6,2023-01-08,32
7,2023-01-09,60
8,2023-01-10,54
9,2023-01-11,60


In [183]:
df_viagens_operador_ext_agg = df_viagens_operador_ext[servico].groupby("data")[["datetime_chegada"]].count().reset_index().rename(columns={"datetime_chegada": "q_viagens_gabarito"})
df_viagens_operador_ext_agg

,data,q_viagens_gabarito
0,2023-01-02,56
1,2023-01-03,54
2,2023-01-04,58
3,2023-01-05,55
4,2023-01-06,56
5,2023-01-07,54
6,2023-01-08,31
7,2023-01-09,58
8,2023-01-10,52
9,2023-01-11,58


In [184]:
df_merged_agg = (df_merged_agg
                 .merge(df_viagem_completa_dev_agg, on="data", how="left")
                 .merge(df_viagens_operador_ext_agg, on="data", how="left"))

df_merged_agg[((df_merged_agg["q_viagens_gabarito_x"] != df_merged_agg["q_viagens_gabarito_y"]) |
               (df_merged_agg["q_viagens_solucao_x"] != df_merged_agg["q_viagens_solucao_y"]))]

,data,q_viagens_gabarito_x,q_viagens_solucao_x,q_viagens_solucao_y,q_viagens_gabarito_y
0,2023-01-02,56,56,58,56
1,2023-01-03,54,53,56,54
2,2023-01-04,58,53,55,58
3,2023-01-05,55,55,59,55
4,2023-01-06,56,56,60,56
5,2023-01-07,54,54,56,54
6,2023-01-08,31,31,32,31
7,2023-01-09,58,58,60,58
8,2023-01-10,52,52,54,52
9,2023-01-11,58,58,60,58


##### Análise Exploratória de Viagens

In [ ]:
#2023-01-03	010	I	A72085	2023-01-03 21:38:00	2023-01-03 21:59:00

In [185]:
servico = "010"
data_ref_23 = "2023-01-03"
start_time = "2023-01-03 21:38:00"
end_time = "2023-01-03 22:30:00"
id_veiculo = "A72085"
viagem_temp_23 = dict()

In [186]:
map_generate(servico, 
             None, 
             data_ref_23, 
             None, 
             start_time, 
             end_time,
             table_id_gps="gps_sppo",
             id_veiculo=id_veiculo,
             dataset_id="20230818_projeto_subsidio_sppo_010",
             mode="dev"
             )


        SELECT
          timestamp_gps,
          'middle' AS status_viagem,
          ST_GEOGPOINT(longitude, latitude) AS posicao_veiculo_geo
        FROM
          `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
        WHERE
          data = '2023-01-03'
          AND servico = '010'
          AND timestamp_gps BETWEEN '2023-01-03 21:38:00' AND '2023-01-03 22:30:00'
          AND id_veiculo = 'A72085'
        


Downloading: 100%|██████████| 147/147 [00:00<00:00, 417.62rows/s]



    SELECT
      *
    FROM
      `rj-smtr-dev.20230818_projeto_subsidio_sppo_010.viagem_planejada`
    WHERE
      servico = '010'
      AND data = '2023-01-03'
      
    


Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.17rows/s]


010 - 
